<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong>
       Importando bibliotecas
    </strong>
</p>

In [1]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Criando planilha de controle de entregas
    </strong>
</p>

In [2]:
controle_path = 'dados/controle_entregas.xlsx'

try:
    wb = Workbook()

    ws = wb.active

    ws.title = 'Entregas'

    ws.append([
        'Data',
        'Estado',
        'Motorista',
        'Entregas_Previstas',
        'Entregas_no_Prazo',
        'Entregas_Realizado'
    ])

    ws.append(['01/01/2026','RJ','Ana','20','6','12'])
    ws.append(['02/01/2026','BA','Joao','40','2','36'])
    ws.append(['03/01/2026','RO','Davi','30','8','12'])
    ws.append(['04/01/2026','SP','Anthony','87','3','84'])
    ws.append(['05/01/2026','MG','Ronaldo','10','1','3'])
    ws.append(['06/01/2026','SP','Anthony','98','11','95'])

    
    wb.save(controle_path)
    print(f"Planilha salva com sucesso em {controle_path}")

except Exception as e:
    print(e)

Planilha salva com sucesso em dados/controle_entregas.xlsx


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Inserindo novos indicadores
    </strong>
</p>

In [3]:
wb = load_workbook(controle_path)
ws = wb.active

#Criando novas colunas
ws['G1'] = "Taxa de Sucesso"
ws['H1'] = "Taxa de Atraso"

for linha in range(2, ws.max_row+1):
    ws[f'G{linha}'] = f'=F{linha}/D{linha}'
    ws[f'H{linha}'] = f'=1-G{linha}'

wb.save(controle_path)
print("Formulas inseridas com sucesso!")
    

Formulas inseridas com sucesso!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Formatando as taxas como percentual
    </strong>
</p>

In [4]:
wb = load_workbook(controle_path)
ws = wb.active

for linha in range(2, ws.max_row+1):
    ws[f'G{linha}'].number_format = '0.00%' 
    ws[f'H{linha}'].number_format = '0.00%'

wb.save(controle_path)
print("Formatação Concluida")

Formatação Concluida


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Resumo gerencial por estado
    </strong>
</p>

In [5]:
df = pd.read_excel(controle_path, sheet_name='Entregas')

# display(df.groupby('Estado').agg( 
#     Total_entregas_Previstas = ('Entregas_Previstas', 'sum'), 
#     Total_entregas_Realizadas = ('Entregas_Realizado', 'sum'), 
#     Taxa_de_Sucesso = ('Taxa de Sucesso', 'median'), 
#     Taxa_de_Atraso = ('Taxa de Atraso', 'median') 
# ))

df['Taxa de Sucesso 2'] = df['Entregas_Realizado'] / df['Entregas_Previstas']

resumo_Estado = df.groupby('Estado').agg({
    "Entregas_Previstas": "sum",
    "Entregas_Realizado": "sum",
    "Taxa de Sucesso": "mean",
    "Taxa de Atraso": "mean"
})

display(resumo_Estado)

wb = load_workbook(controle_path)

if 'Resumo' in wb.sheetnames:
    del wb['Resumo']
ws_resumo = wb.create_sheet("Resumo")

ws_resumo.append([
    "Estado",
    "Entregas_Previstas",
    "Entregas_Realizado",
    "Taxa de Sucesso",
    "Taxa de Atraso"
])

for estado,valores in resumo_Estado.iterrows():
    ws_resumo.append([
        estado,
        valores['Entregas_Previstas'],
        valores['Entregas_Realizado'],
        valores['Taxa de Sucesso'],
        valores['Taxa de Atraso']
    ])

for linha in range(2,ws_resumo.max_row+1):
    ws_resumo[f'D{linha}'].number_format = "0.00%"
    ws_resumo[f'E{linha}'].number_format = "0.00%"

wb.save(controle_path)

,Entregas_Previstas,Entregas_Realizado,Taxa de Sucesso,Taxa de Atraso
Estado,,,,
BA,40,36,NaN,NaN
MG,10,3,NaN,NaN
RJ,20,12,NaN,NaN
RO,30,12,NaN,NaN
SP,185,179,NaN,NaN


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Cálculos de taxas e inserindo no excel
    </strong>
</p>